## Gerando os componentes MLP

In [1]:
import numpy as np
import pandas as pd
import joblib
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.neural_network import MLPRegressor
import os
import json
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

def criar_dataset_simples(series, look_back=5, passo=1):
    """
    Criar dataset SIMPLES - apenas sequência temporal (igual ao LSTM e XGBoost)
    SEM features engineering
    """
    X, y = [], []
    for i in range(len(series) - look_back - passo + 1):
        # Apenas a sequência temporal
        sequencia = series[i:i + look_back]
        target = series[i + look_back + passo - 1]
        
        X.append(sequencia.flatten())
        y.append(target[0])  # target é array, pegar valor
    
    return np.array(X), np.array(y)

def treinar_mlp_componente(nome_comp, passo=1, test_size=0.2, random_state=42):
    """
    Treinar modelo MLP para um componente específico
    Usando APENAS sequências temporais (sem features engineering)
    """
    print(f"🧠 TREINANDO MLP - {nome_comp.upper()} para t+{passo}")
    print("=" * 60)
    
    # 1. Carregar dados
    print(f"📊 1. Carregando dados {nome_comp.upper()}...")
    comp_file = f"{nome_comp.upper()}_component.csv"
    if not Path(comp_file).exists():
        raise FileNotFoundError(f"Arquivo não encontrado: {comp_file}")
    
    df = pd.read_csv(comp_file)
    if nome_comp.upper() not in df.columns:
        raise ValueError(f"Coluna {nome_comp.upper()} não encontrada em {comp_file}")
    
    data = df[nome_comp.upper()].values.reshape(-1, 1)
    print(f"   ✅ Dados carregados: {len(data)} pontos")
    print(f"   📈 Range: [{data.min():.4f}, {data.max():.4f}]")
    print(f"   📊 Std: {np.std(data):.4f}")
    
    # 2. Normalização
    print(f"\n🔧 2. Aplicando normalização...")
    scaler = RobustScaler()
    data_scaled = scaler.fit_transform(data)
    print(f"   ✅ Scaler: RobustScaler")
    print(f"   📈 Range normalizado: [{data_scaled.min():.4f}, {data_scaled.max():.4f}]")
    
    # 3. Criar dataset
    print(f"\n📦 3. Criando dataset...")
    look_back = 5 if passo in [1, 5] else 10  # Regra original
    print(f"   📏 Look_back: {look_back}")
    print(f"   🎯 Passo de previsão: {passo}")
    
    X, y = criar_dataset_simples(data_scaled, look_back, passo)
    print(f"   ✅ Dataset criado:")
    print(f"      X shape: {X.shape} (apenas sequências temporais)")
    print(f"      y shape: {y.shape}")
    print(f"      Features: {X.shape[1]} (= look_back, SEM features engineering)")
    
    # 4. Divisão treino/teste
    print(f"\n✂️ 4. Dividindo dados...")
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state, shuffle=False
    )
    print(f"   ✅ Treino: {X_train.shape[0]} amostras")
    print(f"   ✅ Teste: {X_test.shape[0]} amostras")
    
    # 5. Configurar MLP
    print(f"\n🧠 5. Configurando MLP...")
    
    # Arquitetura adaptativa baseada no look_back
    if look_back == 5:
        # Para look_back menor, arquitetura mais simples
        hidden_layers = (64, 32, 16)
    else:
        # Para look_back maior, arquitetura mais complexa
        hidden_layers = (128, 64, 32, 16)
    
    mlp_params = {
        'hidden_layer_sizes': hidden_layers,
        'activation': 'relu',
        'solver': 'adam',
        'alpha': 0.001,  # Regularização L2
        'learning_rate': 'adaptive',
        'learning_rate_init': 0.001,
        'max_iter': 500,
        'early_stopping': True,
        'validation_fraction': 0.1,
        'n_iter_no_change': 20,
        'random_state': random_state,
        'tol': 1e-4
    }
    
    print(f"   📋 Parâmetros MLP:")
    print(f"      Arquitetura: {hidden_layers}")
    print(f"      Ativação: {mlp_params['activation']}")
    print(f"      Solver: {mlp_params['solver']}")
    print(f"      Learning rate: {mlp_params['learning_rate_init']}")
    print(f"      Regularização: {mlp_params['alpha']}")
    print(f"      Max épocas: {mlp_params['max_iter']}")
    print(f"      Early stopping: {mlp_params['early_stopping']}")
    
    # 6. Treinar modelo
    print(f"\n🚀 6. Treinando modelo...")
    modelo = MLPRegressor(**mlp_params)
    
    # Treinar com verbose para acompanhar
    import time
    inicio = time.time()
    modelo.fit(X_train, y_train)
    tempo_treino = time.time() - inicio
    
    print(f"   ✅ Treinamento concluído em {tempo_treino:.1f}s!")
    print(f"   📊 Iterações realizadas: {modelo.n_iter_}")
    print(f"   📈 Loss final: {modelo.loss_:.6f}")
    
    # 7. Avaliar modelo
    print(f"\n📊 7. Avaliando performance...")
    
    # Previsões
    y_pred_train = modelo.predict(X_train)
    y_pred_test = modelo.predict(X_test)
    
    # Desnormalizar para métricas
    y_train_real = scaler.inverse_transform(y_train.reshape(-1, 1)).flatten()
    y_test_real = scaler.inverse_transform(y_test.reshape(-1, 1)).flatten()
    y_pred_train_real = scaler.inverse_transform(y_pred_train.reshape(-1, 1)).flatten()
    y_pred_test_real = scaler.inverse_transform(y_pred_test.reshape(-1, 1)).flatten()
    
    # Métricas treino
    rmse_train = np.sqrt(mean_squared_error(y_train_real, y_pred_train_real))
    mae_train = mean_absolute_error(y_train_real, y_pred_train_real)
    
    # Métricas teste
    rmse_test = np.sqrt(mean_squared_error(y_test_real, y_pred_test_real))
    mae_test = mean_absolute_error(y_test_real, y_pred_test_real)
    
    print(f"   📈 TREINO:")
    print(f"      RMSE: {rmse_train:.4f}")
    print(f"      MAE:  {mae_train:.4f}")
    print(f"   📈 TESTE:")
    print(f"      RMSE: {rmse_test:.4f}")
    print(f"      MAE:  {mae_test:.4f}")
    
    # Verificar overfitting
    overfitting_ratio = rmse_test / rmse_train
    if overfitting_ratio > 1.5:
        print(f"   ⚠️ Possível overfitting (ratio: {overfitting_ratio:.2f})")
    else:
        print(f"   ✅ Sem overfitting (ratio: {overfitting_ratio:.2f})")
    
    # Verificar convergência
    if modelo.n_iter_ >= mlp_params['max_iter']:
        print(f"   ⚠️ Modelo não convergiu (atingiu max_iter)")
    else:
        print(f"   ✅ Modelo convergiu em {modelo.n_iter_} iterações")
    
    # 8. Salvar arquivos
    print(f"\n💾 8. Salvando arquivos...")
    
    # Criar diretórios
    os.makedirs("modelosMLP", exist_ok=True)
    os.makedirs("scalersMLP", exist_ok=True)
    os.makedirs("configsMLP", exist_ok=True)
    
    # Salvar modelo
    modelo_path = f"modelosMLP/mlp_{nome_comp.lower()}_t{passo}.joblib"
    joblib.dump(modelo, modelo_path)
    print(f"   ✅ Modelo salvo: {modelo_path}")
    
    # Salvar scaler
    scaler_path = f"scalersMLP/scaler_{nome_comp.lower()}_t{passo}.joblib"
    joblib.dump(scaler, scaler_path)
    print(f"   ✅ Scaler salvo: {scaler_path}")
    
    # Salvar configuração
    config = {
        'componente': nome_comp.upper(),
        'passo': passo,
        'look_back': look_back,
        'n_features_total': X.shape[1],
        'n_features_sequencia': look_back,
        'n_features_engineering': 0,  # SEM features engineering
        'scaler_type': 'RobustScaler',
        'model_type': 'MLPRegressor',
        'mlp_params': mlp_params,
        'arquitetura': {
            'hidden_layers': hidden_layers,
            'total_parameters': sum([layer * next_layer for layer, next_layer in 
                                   zip([X.shape[1]] + list(hidden_layers), 
                                       list(hidden_layers) + [1])]),
            'activation': mlp_params['activation'],
            'solver': mlp_params['solver']
        },
        'dataset_info': {
            'total_samples': len(X),
            'train_samples': len(X_train),
            'test_samples': len(X_test),
            'data_range_original': [float(data.min()), float(data.max())],
            'data_range_scaled': [float(data_scaled.min()), float(data_scaled.max())]
        },
        'training_info': {
            'tempo_treino_segundos': float(tempo_treino),
            'iteracoes_realizadas': int(modelo.n_iter_),
            'loss_final': float(modelo.loss_),
            'convergiu': bool(modelo.n_iter_ < mlp_params['max_iter'])
        },
        'performance': {
            'rmse_train': float(rmse_train),
            'mae_train': float(mae_train),
            'rmse_test': float(rmse_test),
            'mae_test': float(mae_test),
            'overfitting_ratio': float(overfitting_ratio)
        }
    }
    
    config_path = f"configsMLP/config_{nome_comp.lower()}_t{passo}.json"
    with open(config_path, 'w', encoding='utf-8') as f:
        json.dump(config, f, indent=2, ensure_ascii=False)
    print(f"   ✅ Config salva: {config_path}")
    
    # 9. Teste de sanidade
    print(f"\n🧪 9. Teste de sanidade...")
    
    # Carregar modelo salvo e testar
    modelo_carregado = joblib.load(modelo_path)
    scaler_carregado = joblib.load(scaler_path)
    
    # Teste com uma amostra
    amostra_teste = X_test[:5]  # 5 amostras
    pred_teste = modelo_carregado.predict(amostra_teste)
    pred_teste_real = scaler_carregado.inverse_transform(pred_teste.reshape(-1, 1)).flatten()
    
    print(f"   ✅ Modelo carregado e testado:")
    print(f"      Amostras teste: {len(amostra_teste)}")
    print(f"      Previsões (normalizado): {pred_teste[:3]}")
    print(f"      Previsões (real): {pred_teste_real[:3]}")
    print(f"      Variabilidade: {np.std(pred_teste_real):.4f}")
    
    print(f"\n🎉 TREINAMENTO MLP CONCLUÍDO COM SUCESSO!")
    print(f"   📁 Arquivos salvos:")
    print(f"      - {modelo_path}")
    print(f"      - {scaler_path}")
    print(f"      - {config_path}")
    
    return {
        'modelo': modelo,
        'scaler': scaler,
        'config': config,
        'performance': {
            'rmse_test': rmse_test,
            'mae_test': mae_test,
            'overfitting_ratio': overfitting_ratio,
            'tempo_treino': tempo_treino,
            'convergiu': modelo.n_iter_ < mlp_params['max_iter']
        }
    }

def treinar_todos_componentes_mlp(passos=[1, 5, 7, 30]):
    """
    Treinar modelos MLP para todos os componentes D1, D2, D3
    em todos os horizontes de previsão
    """
    print("🧠 TREINAMENTO COMPLETO - MLP SIMPLES")
    print("=" * 70)
    print(f"📋 Componentes: D1, D2, D3")
    print(f"🎯 Horizontes: {passos}")
    print(f"🔧 Abordagem: Apenas sequências temporais (sem features)")
    print(f"🧠 Modelo: Multi-Layer Perceptron (MLP)")
    print("=" * 70)
    
    resultados = {}
    componentes = ['d1', 'd2', 'd3']
    
    total_modelos = len(componentes) * len(passos)
    contador = 0
    
    import time
    inicio_total = time.time()
    
    for comp in componentes:
        resultados[comp] = {}
        
        for passo in passos:
            contador += 1
            print(f"\n{'🟢' * contador}{'⚪' * (total_modelos - contador)} PROGRESSO: {contador}/{total_modelos}")
            print(f"🎯 Treinando {comp.upper()} para t+{passo}")
            
            try:
                resultado = treinar_mlp_componente(comp, passo)
                resultados[comp][f't+{passo}'] = resultado['performance']
                
                convergiu = "✅ Convergiu" if resultado['performance']['convergiu'] else "⚠️ Não convergiu"
                print(f"✅ {comp.upper()} t+{passo} - SUCESSO (RMSE: {resultado['performance']['rmse_test']:.4f}) - {convergiu}")
                
            except Exception as e:
                print(f"❌ {comp.upper()} t+{passo} - ERRO: {e}")
                resultados[comp][f't+{passo}'] = {'erro': str(e)}
    
    tempo_total = time.time() - inicio_total
    
    # Resumo final
    print(f"\n{'=' * 70}")
    print("📊 RESUMO FINAL DO TREINAMENTO MLP")
    print("=" * 70)
    print(f"⏱️ Tempo total: {tempo_total/60:.1f} minutos")
    
    print(f"{'Componente':<12} {'Horizonte':<10} {'RMSE':<8} {'MAE':<8} {'Convergiu':<10} {'Status':<10}")
    print("-" * 80)
    
    sucessos = 0
    falhas = 0
    nao_convergiu = 0
    
    for comp in componentes:
        for passo in passos:
            key = f't+{passo}'
            if key in resultados[comp]:
                if 'erro' in resultados[comp][key]:
                    print(f"{comp.upper():<12} {key:<10} {'N/A':<8} {'N/A':<8} {'N/A':<10} {'ERRO':<10}")
                    falhas += 1
                else:
                    perf = resultados[comp][key]
                    rmse = perf['rmse_test']
                    mae = perf['mae_test']
                    convergiu = "Sim" if perf['convergiu'] else "Não"
                    if not perf['convergiu']:
                        nao_convergiu += 1
                    
                    print(f"{comp.upper():<12} {key:<10} {rmse:<8.4f} {mae:<8.4f} {convergiu:<10} {'OK':<10}")
                    sucessos += 1
    
    print(f"\n📊 ESTATÍSTICAS:")
    print(f"   ✅ Sucessos: {sucessos}")
    print(f"   ❌ Falhas: {falhas}")
    print(f"   ⚠️ Não convergiram: {nao_convergiu}")
    print(f"   📈 Taxa de sucesso: {(sucessos/(sucessos+falhas)*100):.1f}%")
    
    if nao_convergiu > 0:
        print(f"\n💡 DICA: {nao_convergiu} modelos não convergiram.")
        print(f"   Considere aumentar max_iter ou ajustar learning_rate.")
    
    # Salvar resumo
    resumo_path = "configsMLP/resumo_treinamento_mlp.json"
    resumo = {
        'data_treinamento': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S'),
        'abordagem': 'MLP simples - apenas sequências temporais',
        'modelo_tipo': 'Multi-Layer Perceptron (MLPRegressor)',
        'componentes': componentes,
        'horizontes': passos,
        'total_modelos': total_modelos,
        'tempo_total_minutos': tempo_total / 60,
        'sucessos': sucessos,
        'falhas': falhas,
        'nao_convergiram': nao_convergiu,
        'resultados': resultados
    }
    
    os.makedirs("configsMLP", exist_ok=True)
    with open(resumo_path, 'w', encoding='utf-8') as f:
        json.dump(resumo, f, indent=2, ensure_ascii=False)
    
    print(f"\n💾 Resumo salvo: {resumo_path}")
    
    if sucessos == total_modelos:
        print(f"\n🎉 TODOS OS MODELOS MLP TREINADOS COM SUCESSO!")
    else:
        print(f"\n⚠️ Alguns modelos falharam. Verifique os logs acima.")
    
    return resultados

def comparar_arquiteturas_mlp(nome_comp='d1', passo=1):
    """
    Função para testar diferentes arquiteturas MLP e encontrar a melhor
    """
    print(f"🔬 TESTE DE ARQUITETURAS MLP - {nome_comp.upper()} t+{passo}")
    print("=" * 60)
    
    # Arquiteturas a testar
    arquiteturas = {
        'simples': (32, 16),
        'media': (64, 32, 16),
        'complexa': (128, 64, 32, 16),
        'profunda': (128, 128, 64, 64, 32, 16),
        'larga': (256, 128, 64)
    }
    
    # Carregar dados
    comp_file = f"{nome_comp.upper()}_component.csv"
    df = pd.read_csv(comp_file)
    data = df[nome_comp.upper()].values.reshape(-1, 1)
    
    scaler = RobustScaler()
    data_scaled = scaler.fit_transform(data)
    
    look_back = 5 if passo in [1, 5] else 10
    X, y = criar_dataset_simples(data_scaled, look_back, passo)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)
    
    resultados_arq = {}
    
    for nome_arq, hidden_layers in arquiteturas.items():
        print(f"\n🧠 Testando arquitetura '{nome_arq}': {hidden_layers}")
        
        try:
            modelo = MLPRegressor(
                hidden_layer_sizes=hidden_layers,
                activation='relu',
                solver='adam',
                alpha=0.001,
                learning_rate='adaptive',
                learning_rate_init=0.001,
                max_iter=300,
                early_stopping=True,
                validation_fraction=0.1,
                n_iter_no_change=15,
                random_state=42,
                tol=1e-4
            )
            
            inicio = time.time()
            modelo.fit(X_train, y_train)
            tempo_treino = time.time() - inicio
            
            # Avaliar
            y_pred_test = modelo.predict(X_test)
            y_test_real = scaler.inverse_transform(y_test.reshape(-1, 1)).flatten()
            y_pred_real = scaler.inverse_transform(y_pred_test.reshape(-1, 1)).flatten()
            
            rmse = np.sqrt(mean_squared_error(y_test_real, y_pred_real))
            mae = mean_absolute_error(y_test_real, y_pred_real)
            
            # Calcular número de parâmetros
            n_params = sum([layer * next_layer for layer, next_layer in 
                           zip([X.shape[1]] + list(hidden_layers), 
                               list(hidden_layers) + [1])])
            
            resultados_arq[nome_arq] = {
                'rmse': rmse,
                'mae': mae,
                'tempo_treino': tempo_treino,
                'n_parametros': n_params,
                'convergiu': modelo.n_iter_ < 300,
                'iteracoes': modelo.n_iter_,
                'loss_final': modelo.loss_
            }
            
            status = "✅ Convergiu" if modelo.n_iter_ < 300 else "⚠️ Não convergiu"
            print(f"   RMSE: {rmse:.4f} | MAE: {mae:.4f} | Params: {n_params} | {status}")
            
        except Exception as e:
            print(f"   ❌ Erro: {e}")
            resultados_arq[nome_arq] = {'erro': str(e)}
    
    # Encontrar melhor arquitetura
    print(f"\n🏆 RANKING DAS ARQUITETURAS:")
    print(f"{'Arquitetura':<12} {'RMSE':<8} {'MAE':<8} {'Params':<8} {'Tempo':<8} {'Status'}")
    print("-" * 70)
    
    arquiteturas_validas = {k: v for k, v in resultados_arq.items() if 'erro' not in v}
    ranking = sorted(arquiteturas_validas.items(), key=lambda x: x[1]['rmse'])
    
    for i, (nome, resultado) in enumerate(ranking):
        emoji = "🥇" if i == 0 else "🥈" if i == 1 else "🥉" if i == 2 else "  "
        status = "Conv." if resultado['convergiu'] else "N.Conv."
        print(f"{emoji} {nome:<10} {resultado['rmse']:<8.4f} {resultado['mae']:<8.4f} "
              f"{resultado['n_parametros']:<8} {resultado['tempo_treino']:<8.1f}s {status}")
    
    if ranking:
        melhor = ranking[0]
        print(f"\n🎯 MELHOR ARQUITETURA: {melhor[0]} - {arquiteturas[melhor[0]]}")
    
    return resultados_arq

# Interface de uso
print("🧠 TREINAMENTO MLP PARA COMPONENTES")
print("=" * 50)
print("📋 Funções disponíveis:")
print()
print("🔹 treinar_mlp_componente('d1', passo=1)")
print("   → Treina um modelo MLP específico")
print()
print("🔹 treinar_todos_componentes_mlp([1, 5, 7, 30])")
print("   → Treina todos os modelos MLP para todos os horizontes")
print()
print("🔹 comparar_arquiteturas_mlp('d1', passo=1)")
print("   → Testa diferentes arquiteturas para encontrar a melhor")
print()
print("💡 CARACTERÍSTICAS DOS MODELOS MLP:")
print("   • Apenas sequências temporais (consistente com LSTM/XGBoost)")
print("   • Look_back: 5 para t+1,t+5 | 10 para demais")
print("   • Scaler: RobustScaler")
print("   • Modelo: MLPRegressor com arquitetura adaptativa")
print("   • Early stopping e regularização L2")
print("   • Otimizador: Adam com learning rate adaptativo")
print()
print("🎯 EXEMPLO DE USO:")
print("   # Treinar um modelo específico")
print("   resultado = treinar_mlp_componente('d1', 1)")
print()
print("   # Treinar todos os modelos")
print("   resultados = treinar_todos_componentes_mlp([1, 5, 7, 30])")
print()
print("   # Comparar arquiteturas")
print("   melhores = comparar_arquiteturas_mlp('d1', 1)")
print("=" * 50)

🧠 TREINAMENTO MLP PARA COMPONENTES
📋 Funções disponíveis:

🔹 treinar_mlp_componente('d1', passo=1)
   → Treina um modelo MLP específico

🔹 treinar_todos_componentes_mlp([1, 5, 7, 30])
   → Treina todos os modelos MLP para todos os horizontes

🔹 comparar_arquiteturas_mlp('d1', passo=1)
   → Testa diferentes arquiteturas para encontrar a melhor

💡 CARACTERÍSTICAS DOS MODELOS MLP:
   • Apenas sequências temporais (consistente com LSTM/XGBoost)
   • Look_back: 5 para t+1,t+5 | 10 para demais
   • Scaler: RobustScaler
   • Modelo: MLPRegressor com arquitetura adaptativa
   • Early stopping e regularização L2
   • Otimizador: Adam com learning rate adaptativo

🎯 EXEMPLO DE USO:
   # Treinar um modelo específico
   resultado = treinar_mlp_componente('d1', 1)

   # Treinar todos os modelos
   resultados = treinar_todos_componentes_mlp([1, 5, 7, 30])

   # Comparar arquiteturas
   melhores = comparar_arquiteturas_mlp('d1', 1)


In [2]:
resultados = treinar_todos_componentes_mlp([1, 5, 7, 30])

🧠 TREINAMENTO COMPLETO - MLP SIMPLES
📋 Componentes: D1, D2, D3
🎯 Horizontes: [1, 5, 7, 30]
🔧 Abordagem: Apenas sequências temporais (sem features)
🧠 Modelo: Multi-Layer Perceptron (MLP)

🟢⚪⚪⚪⚪⚪⚪⚪⚪⚪⚪⚪ PROGRESSO: 1/12
🎯 Treinando D1 para t+1
🧠 TREINANDO MLP - D1 para t+1
📊 1. Carregando dados D1...
   ✅ Dados carregados: 7878 pontos
   📈 Range: [-7.7183, 8.8608]
   📊 Std: 1.3112

🔧 2. Aplicando normalização...
   ✅ Scaler: RobustScaler
   📈 Range normalizado: [-5.4508, 6.2790]

📦 3. Criando dataset...
   📏 Look_back: 5
   🎯 Passo de previsão: 1
   ✅ Dataset criado:
      X shape: (7873, 5) (apenas sequências temporais)
      y shape: (7873,)
      Features: 5 (= look_back, SEM features engineering)

✂️ 4. Dividindo dados...
   ✅ Treino: 6298 amostras
   ✅ Teste: 1575 amostras

🧠 5. Configurando MLP...
   📋 Parâmetros MLP:
      Arquitetura: (64, 32, 16)
      Ativação: relu
      Solver: adam
      Learning rate: 0.001
      Regularização: 0.001
      Max épocas: 500
      Early stopping